## Case #004: The Midnight Masquerade Murder

On October 31, 1987, at a Coconut Grove mansion masked ball, Leonard Pierce was found dead in the garden. Can you piece together all the clues to expose the true murderer?

### Objectives

1. Reveal the true murderer of this complex case.

<img src="./images/Screenshot 2026-02-06 090857.png">

- Looking at the full description of the crime scene

In [ ]:
SELECT id, description
FROM crime_scene
WHERE date = 19871031
AND location LIKE '%Coconut Grove%'

<img src="./images/Screenshot 2026-02-06 091156.png">

- Now, let's see what did the witnesses said

In [ ]:
SELECT p.name, ws.clue
FROM person AS p
JOIN witness_statements AS ws
ON p.id = ws.witness_id
WHERE ws.crime_scene_id = (
    SELECT id
    FROM crime_scene
    WHERE date = 19871031
    AND location LIKE '%Coconut Grove%'
)

<img src="./images/Screenshot 2026-02-06 094521.png">

In [ ]:
SELECT COUNT(*)
FROM hotel_checkins
WHERE hotel_name = 'The Grand Regency'
    AND room_number = 707
    AND check_in_date = 19871030

- Executing this, would result that there are `10` entries. This does not narrow it down, so we need to dig deeper

In [ ]:
SELECT p.name, sr.note
FROM person AS p
JOIN hotel_checkins AS hc
ON p.id = hc.person_id
JOIN surveillance_records AS sr
ON hc.id = sr.hotel_checkin_id
WHERE hc.hotel_name = 'The Grand Regency'
    AND hc.room_number = 707
    AND hc.check_in_date = 19871030
    AND sr.note IS NOT NULL

<img src="./images/Screenshot 2026-02-06 095548.png">

- Looking at the result, `Antonio Rossi's` call seems suspicious enough. Let's see who is the person on the other side

In [ ]:
SELECT p.name AS recipient_name,
    pr.call_date,
    pr.call_time,
    pr.note
FROM person AS p
JOIN phone_records AS pr
ON p.id = pr.recipient_id 
WHERE pr.caller_id = (
    SELECT id
    FROM person
    WHERE name = 'Antonio Rossi'
)

<img src="./images/Screenshot 2026-02-06 100326.png">

- From his conversation with `Victor DiMarco`, it seems that the latter is the killer. Now, we need to get his interview

In [ ]:
SELECT confession
FROM final_interviews
WHERE person_id = (
    SELECT id 
    FROM person 
    WHERE name = 'Victor DiMarco'
)

<img src="./images/Screenshot 2026-02-06 100701.png">

- Apparently, this guy is just a middleman. And applying the previous code to `Antonio Rossi` yields that he is also not the killer

<img src="./images/Screenshot 2026-02-06 100850.png">

- Now, let's see the call records of `Victor DiMarco` since he is still part of the crime

In [ ]:
SELECT p.name AS recipient_name,
    pr.call_date,
    pr.call_time,
    pr.note
FROM person AS p
JOIN phone_records AS pr
ON p.id = pr.caller_id 
WHERE pr.recipient_id = (
    SELECT id
    FROM person
    WHERE name = 'Victor DiMarco'
)

<img src="./images/Screenshot 2026-02-06 101118.png">

- Looking at the call logs (as a recipient), we can see the true killer, but the name of that person is `Unknown`. However, the `Lambo` is a key evidence here. Since the operation was succesful, we need to look for people who now owns a `Lamborghini`

In [ ]:
SELECT COUNT(*)
FROM person AS p
JOIN vehicle_registry AS vr
    ON p.id = vr.person_id
WHERE vr.car_make LIKE '%Lambo%'

- Apparently, there are `19` owners of a `Lamborghini` car. We need to narrow it down. Remember that `Antonio Rossi` told `Victor DiMarco` about a `carpenter`. Let's include this parameter

In [ ]:
SELECT DISTINCT p.name
FROM person AS p
JOIN vehicle_registry AS vr
    ON p.id = vr.person_id
WHERE vr.car_make LIKE '%Lambo%'
    AND LOWER(p.occupation) = 'carpenter'

<img src="./images/Screenshot 2026-02-06 110025.png">

- Now it gives us only one person. Let's check his confession

In [ ]:
SELECT confession
FROM final_interviews
WHERE person_id = (
    SELECT id
    FROM person
    WHERE name = 'Marco Santos'
)

<img src="./images/Screenshot 2026-02-06 110159.png">

- It's confirmed that `Marco Santos` is the killer

<img src="./images/Screenshot 2026-02-06 110437.png">